# iris center - deeplab v3+ ver

# import

In [1]:
import keras
from keras import backend as K

def load_modules():
    import sys
    import numpy as np
    import warnings
    
    sys.path.append("../../../")
    warnings.filterwarnings("ignore")
    np.set_printoptions(precision=6, suppress=True)
    
    %run ../../../ds/icontour/npz/gen.py
    %run ../../../ac/visualizer/plotter.py
    %run ../../../ai/model/deeplab/iris.py
    %run ../../../ai/libs/keras/callbacks/history.py
    %run ../../../ai/libs/keras/callbacks/checkpoint.py
    %run ../../../ai/libs/keras/callbacks/stopper.py
    
load_modules()

Using TensorFlow backend.


In [2]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

# backbone model

In [3]:
INPUT_SHAPE=(112, 112, 3)

icm = make_iris_deeplab_v3(input_shape=INPUT_SHAPE, classes=1, alpha=1.)
icm.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 112, 112, 3)  0                                            
__________________________________________________________________________________________________
Conv (Conv2D)                   (None, 56, 56, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv_BN (BatchNormalization)    (None, 56, 56, 32)   128         Conv[0][0]                       
__________________________________________________________________________________________________
Conv_Relu6 (Activation)         (None, 56, 56, 32)   0           Conv_BN[0][0]                    
__________________________________________________________________________________________________
expanded_c

# callback

In [4]:
chk_path = "/home/chy/archive-model/incubator/iris-contour/iris-{epoch:02d}-{val_loss:.7f}.hdf5"

history = TimeHistory()
checkpoint = model_checkpoint(chk_path)
stopper = EarlyStopping(monitor='val_loss', 
                        min_delta=0, 
                        patience=5, 
                        verbose=0, 
                        mode='auto', 
                        baseline=None, 
                        restore_best_weights=True)

callbacks = [history, checkpoint, stopper]

# compile

In [5]:
BATCH_SIZE = 64

def pixel_wise_softmax_loss(y_true, y_pred):
    # blank output 다른 원인 중 하나 (UNet 기준)
    # https://discuss.pytorch.org/t/unet-implementation/426/23?u=ptrblck
     
    # deeplab single object blank 현상 (배경과 대상 객체 비율 불균형)
    # https://github.com/tensorflow/models/issues/3739#issuecomment-402583877
    # https://github.com/rishizek/tensorflow-deeplab-v3/issues/7
    
    # keras 팀에 따르면 num_batch x flatten x prob 형태로 계산하면 된다고 함 
    # https://github.com/keras-team/keras/issues/1169
    
    # 이미지 상에서 레이블 객체의 크기가 작고, 레이블 개수가 작은 경우 
    # 일반적인 pixel wise softmax는 모두 배경 레이블로 수렴하려 함.
    # nb_classes = K.int_shape(y_pred)[-1]
    # p = K.reshape(y_pred, shape=(BATCH_SIZE, 112*112, 1))
    
    t = K.reshape(y_true, shape=(BATCH_SIZE, 112*112, 1))
    t = K.cast(t, dtype=np.float32)
    p = K.reshape(y_pred, shape=(BATCH_SIZE, 112*112, 1))
    p = K.cast(p, dtype=np.float32)
    
    return keras.losses.categorical_crossentropy(t, p)
    
    # weighted + background ignore 실험   # 뭔가 계산을 잘 못 했나?? 체크할 것
    # y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
    # y_pred =  K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # loss = y_true * K.log(y_pred) * [0, 100, 1]   # 눈 홍채 영역이 1/100 정도 되는 듯
    # loss = -K.sum(loss, -1)
    
    return loss

In [6]:
icm.compile(loss=pixel_wise_softmax_loss, optimizer='adam', metrics=['acc'])

In [7]:
EXP_CODE = "pilot"
NUM_EPOCH = 5
BATCH_SIZE = 64

# model save

In [8]:
model_json = icm.to_json()
with open("./iris-{}.json".format(EXP_CODE), "w") as json_file : 
    json_file.write(model_json)

# train report

#### Table
|exp|aug|rot|type|comment|
|---|---|---|---|---|
|pilot|x|x|타원 파라미터 추론|mob-v2 기반 실패 (평균값을 예측함)|
|pilot|x|x|픽셀 점들 추론|..|
|pilot|x|x|sementic segmentation|deep lab v3+ backbone mobv2|
|pilot|x|x|deeplab v3+|scelar 영역도 고려하여 classmap 생성|
|pilot|x|x|deeplab v3+|dense 붙여서 중앙 예측|



# train

In [9]:
# npz_path = "/media/chy/1326657F605F16F2/processed/iris-contour-npz/"
# npz_path = "/home/chy/archive-data/processed/iris-contour-npz/unity"
npz_path = "/home/chy/archive-data/processed/iris-contour-npz/unity-partial"
# npz_path = "/home/chy/archive-data/processed/iris-contour-npz/vc"
# npz_path = "/home/chy/archive-data/processed/iris-contour-npz/vc-partial"

gen_train = IrisContourGenerator(npz_base_path=npz_path, 
                                 batch_size=BATCH_SIZE, 
                                 purpose=Purpose.TRAIN,
                                 is_ellipse_mode=False,
                                 use_aug=False)

gen_valid = IrisContourGenerator(npz_base_path=npz_path, 
                                 batch_size=BATCH_SIZE, 
                                 purpose=Purpose.VALID,
                                 is_ellipse_mode=False,
                                 use_aug=False)

*** meta verify complete [Unknown] ***
*** meta verify complete [Unknown] ***


In [10]:
icm.fit_generator(generator=gen_train,
                  validation_data=gen_valid,
                  callbacks=callbacks,
                  epochs=NUM_EPOCH,
                  workers=16, 
                  use_multiprocessing=True,
                  shuffle=True)

Epoch 1/5
560/560 [==============================] - 726s 1s/step - loss: 2.3373e-06 - acc: 0.3106 - val_loss: 2.3141e-06 - val_acc: 0.3227
Epoch 2/5
255/560 [============>.................] - ETA: 4:49 - loss: 2.3324e-06 - acc: 0.3110

Process ForkPoolWorker-64:
Process ForkPoolWorker-59:
Process ForkPoolWorker-57:
Process ForkPoolWorker-48:
Process ForkPoolWorker-58:
Process ForkPoolWorker-54:
Process ForkPoolWorker-52:
Process ForkPoolWorker-55:
Process ForkPoolWorker-49:
Process ForkPoolWorker-53:
Process ForkPoolWorker-44:
Process ForkPoolWorker-38:
Process ForkPoolWorker-63:
Process ForkPoolWorker-45:
Process ForkPoolWorker-36:
Process ForkPoolWorker-46:
Process ForkPoolWorker-61:
Process ForkPoolWorker-35:
Process ForkPoolWorker-50:
Process ForkPoolWorker-40:
Process ForkPoolWorker-60:
Process ForkPoolWorker-34:
Process ForkPoolWorker-42:
Process ForkPoolWorker-56:
Process ForkPoolWorker-37:
Process ForkPoolWorker-41:
Process ForkPoolWorker-47:
Process ForkPoolWorker-51:
Process ForkPoolWorker-33:
Process ForkPoolWorker-39:
Process ForkPoolWorker-43:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent 

  File "/opt/intel/intelpython3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/intel/intelpython3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/intel/intelpython3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/intel/intelpython3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/opt/intel/intelpython3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/intel/intelpython3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/intel/intelpython3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/intel/intelpython3/lib/python3.6/multiprocessing/process

  File "/home/chy/devs/chyenv/lib/python3.6/site-packages/keras/utils/data_utils.py", line 401, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "/opt/intel/intelpython3/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/opt/intel/intelpython3/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/opt/intel/intelpython3/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/chy/devs/chyenv/lib/python3.6/site-packages/keras/utils/data_utils.py", line 401, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "/opt/intel/intelpython3/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/chy/devs/chyenv/lib/python3.6/site-packages/keras/utils/data_utils.py", line 401, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "/home/chy/devs/chyenv/lib/python3.6/site-packages/keras/utils/data_utils.py", line 401, 

  File "../../../ds/core/npz/gen.py", line 91, in load_npz_cache
    self.cache_npz = clone_npz(npz)
KeyboardInterrupt
KeyboardInterrupt
  File "../../../ac/images/makes/class_map.py", line 18, in make_class_map_from_ellipse
    is_in = is_inside(px, py, cx, cy, width, height, phi)
  File "/home/chy/dev-bench/auto-catch/ds/icontour/npz/gen.py", line 48, in transform_label
    labels.append(make_class_map_from_ellipse(self.out_shape, ellipse_param))
  File "/home/chy/dev-bench/auto-catch/ds/icontour/npz/gen.py", line 55, in transform_image
    images = apply(self.preprocess_img, items)
  File "/home/chy/dev-bench/auto-catch/ds/icontour/npz/gen.py", line 48, in transform_label
    labels.append(make_class_map_from_ellipse(self.out_shape, ellipse_param))
  File "/opt/intel/intelpython3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
  File "/home/chy/dev-bench/auto-catch/ds/icontour/npz/gen.py", line 48, in transform_l

KeyboardInterrupt: 

# test

In [ ]:
gen_valid = IrisContourGenerator(npz_base_path=npz_path, 
                                  batch_size=BATCH_SIZE, 
                                  purpose=Purpose.TEST,
                                  is_ellipse_mode=False,
                                  use_aug=True)